In [4]:
!python -m spacy download en_core_web_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.7/400.7 MB 2.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [5]:
import pandas as pd
import numpy as np
import re
import string
from sklearn.model_selection import train_test_split
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import os
import seaborn as sns
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import spacy
import csv
from sklearn.manifold import TSNE
from sklearn import linear_model
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import clear_output

nlp = spacy.load('en_core_web_lg')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [6]:
#make train/test split

df_sample = pd.read_csv("/content/yelp_cleaned_full_test.csv")
X_train, X_test, y_train, y_test = train_test_split(
    df_sample['clean_text'],
    df_sample['label'],
    test_size=0.2,
    random_state=42,
    stratify=df_sample['label']
)

#Confirm sizes
print("Training samples:", len(X_train))
print("Testing samples:", len(X_test))

Training samples: 40000
Testing samples: 10000


In [8]:
#Testing BOW and TFIDF

# BoW Vectorization
bow_vectorizer = CountVectorizer(max_features = 1000000, ngram_range=(1,2), stop_words='english')
X_train_bow = bow_vectorizer.fit_transform(X_train)
# Handle NaN values in X_test before transforming
X_test = X_test.fillna('')  # Replace NaN with empty string
X_test_bow = bow_vectorizer.transform(X_test)

# TF-IDF Vectorization
tfidf_vectorizer = TfidfVectorizer(max_features=1000000, ngram_range=(1,2), stop_words='english')
#Stop words should have already been removed during cleaning process
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test = X_test.fillna('')  # Replace NaN with empty string
X_test_tfidf = tfidf_vectorizer.transform(X_test)

print("BoW shape:", X_train_bow.shape)
print("TF-IDF shape:", X_train_tfidf.shape)

BoW shape: (40000, 1000000)
TF-IDF shape: (40000, 1000000)


In [9]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Creating model
model = LogisticRegression()  #establishing Log Model
model.fit(X_train_tfidf, y_train)  #training model

#Evaluate and getting score of model
tfidf_accuracy = model.score(X_test_tfidf, y_test) #Fitting score to test models
print(f"Model Accuracy: {tfidf_accuracy}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Model Accuracy: 0.5538


In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

#Creating BOW
model = LogisticRegression()  # Initialize model again
model.fit(X_train_bow, y_train)  #Train model again

#Getting score of trained model
bow_accuracy = model.score(X_test_bow, y_test)  #get accuracy of model, use X_test_bow here
print(f"TFIDF Accuracy: {tfidf_accuracy}")
print(f"BOW Accuracy: {bow_accuracy}")

TFIDF Accuracy: 0.5538
BOW Accuracy: 0.53


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [11]:
#Print Classifcation report for both TFIDF and BOW
from sklearn.metrics import classification_report


y_pred_tfidf = model.predict(X_test_tfidf)
y_pred_bow = model.predict(X_test_bow)
print(classification_report(y_test, y_pred_tfidf))
print(classification_report(y_test, y_pred_bow))

              precision    recall  f1-score   support

           0       0.40      0.96      0.56      2000
           1       0.47      0.05      0.09      2000
           2       0.53      0.20      0.29      2000
           3       0.44      0.30      0.36      2000
           4       0.51      0.74      0.61      2000

    accuracy                           0.45     10000
   macro avg       0.47      0.45      0.38     10000
weighted avg       0.47      0.45      0.38     10000

              precision    recall  f1-score   support

           0       0.66      0.72      0.69      2000
           1       0.48      0.42      0.44      2000
           2       0.44      0.44      0.44      2000
           3       0.43      0.43      0.43      2000
           4       0.62      0.65      0.63      2000

    accuracy                           0.53     10000
   macro avg       0.52      0.53      0.53     10000
weighted avg       0.52      0.53      0.53     10000



In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Initialize and train the model
lr_model = LogisticRegression(multi_class='multinomial', solver='saga', max_iter=1000)
lr_model.fit(X_train_tfidf, y_train)

# Predictions
y_pred_lr = lr_model.predict(X_test_tfidf)

# Evaluation
print("Logistic Regression Accuracy:", accuracy_score(y_test, y_pred_lr))
print("\nClassification Report:\n", classification_report(y_test, y_pred_lr, digits=3))


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Logistic Regression Accuracy: 0.5533

Classification Report:
               precision    recall  f1-score   support

           0      0.663     0.716     0.689      2000
           1      0.501     0.455     0.477      2000
           2      0.472     0.479     0.476      2000
           3      0.472     0.469     0.470      2000
           4      0.645     0.648     0.647      2000

    accuracy                          0.553     10000
   macro avg      0.551     0.553     0.552     10000
weighted avg      0.551     0.553     0.552     10000

